<a href="https://colab.research.google.com/github/naman065/EE-655-Course-Project/blob/main/Course_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision einops wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
!git clone https://github.com/saurabhya/FCCNs.git



Cloning into 'FCCNs'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 105 (delta 56), reused 77 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (105/105), 1.38 MiB | 25.69 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [3]:
import os
os.chdir('/content/FCCNs')  # Change working directory to FCCNs


In [43]:
import torch

from networks import ResNet, Bottleneck  # Import the ResNet model from networks_new.py


In [16]:
!unzip /content/resnet50_best.zip -d /content/


Archive:  /content/resnet50_best.zip
   creating: /content/resnet50_best/
   creating: /content/resnet50_best/archive/
   creating: /content/resnet50_best/archive/data/
  inflating: /content/resnet50_best/archive/data/0  
  inflating: /content/resnet50_best/archive/data/1  
  inflating: /content/resnet50_best/archive/data/10  
  inflating: /content/resnet50_best/archive/data/100  
  inflating: /content/resnet50_best/archive/data/1000  
  inflating: /content/resnet50_best/archive/data/1001  
  inflating: /content/resnet50_best/archive/data/1002  
  inflating: /content/resnet50_best/archive/data/1003  
  inflating: /content/resnet50_best/archive/data/1004  
  inflating: /content/resnet50_best/archive/data/1005  
  inflating: /content/resnet50_best/archive/data/1006  
  inflating: /content/resnet50_best/archive/data/1007  
  inflating: /content/resnet50_best/archive/data/1008  
  inflating: /content/resnet50_best/archive/data/1009  
  inflating: /content/resnet50_best/archive/data/101  
 

In [18]:
import os
print(os.listdir('/content/resnet50_best'))


['archive']


In [44]:
model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes=1000)

In [45]:
import torch

checkpoint_path = '/content/resnet50_best.tar'
checkpoint = torch.load(checkpoint_path, map_location='cpu')
# Check the keys in the checkpoint; typically, the state_dict is under a key, e.g., "state_dict":
if 'state_dict' in checkpoint:
    state_dict = checkpoint['state_dict']
else:
    state_dict = checkpoint

model.load_state_dict(state_dict, strict=False)
model.eval()  # Set to eval mode
print("Pretrained complex-valued ResNet50 loaded successfully!")


Pretrained complex-valued ResNet50 loaded successfully!


In [46]:
import torch.nn as nn
import complexnn as comp  # from the repository

class DetectionHead(nn.Module):
    def __init__(self, in_channels, num_anchors=1):
        """
        in_channels: Number of channels from the backbone feature map.
        num_anchors: Number of predictions per spatial location (set to 1 for simplicity).
        Output: 4 channels per anchor corresponding to [x, y, h, w] (complex values).
        """
        super(DetectionHead, self).__init__()
        # 1x1 complex convolution: output channels = num_anchors * 4
        self.conv = comp.ComplexConv2d(in_channels, num_anchors * 4, kernel_size=1)

    def forward(self, x):
        # x: complex feature map (B, C, H, W)
        out = self.conv(x)
        # For simplicity, perform adaptive average pooling to collapse spatial dimensions:
        out = nn.functional.adaptive_avg_pool2d(out, (1,1))
        out = out.view(out.size(0), 4)  # Now shape is (B, 4)
        return out


In [56]:
class ComplexObjectDetector(nn.Module):
    def __init__(self, backbone, detection_head):
        super(ComplexObjectDetector, self).__init__()
        self.backbone = backbone
        self.detection_head = detection_head
        # Remove or bypass the original final layer.
        # Here, we replace the fully connected layer with an identity.
        self.backbone.fc = nn.Identity()

    def forward(self, x):
        # x: complex image tensor
        features = self.backbone(x)  # Expecting a feature map
        bbox_pred = self.detection_head(features)  # Complex output: shape (B, 4)
        return bbox_pred

# Assume the backbone outputs, for example, 512 channels (adjust based on your network).
detection_head = DetectionHead(in_channels=2048)  # Change if your backbone outputs a different number.
detector = ComplexObjectDetector(model, detection_head)
detector.eval()  # For inference (set to train() during training)


ComplexObjectDetector(
  (backbone): ResNet(
    (conv1): ComplexConv2d(
      (conv_real): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (conv_imag): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    )
    (bn1): ComplexNaiveBatchNorm2d(
      (real_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (imag_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (prelu): CPReLU(
      (real_prelu): PReLU(num_parameters=1)
      (imag_prelu): PReLU(num_parameters=1)
    )
    (maxpool): ComplexMaxPool2d(
      (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): ComplexConv2d(
          (conv_real): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (conv_imag): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=Fa

In [59]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
detector = detector.to(device)


In [49]:
class ComplexBBoxLoss(nn.Module):
    def __init__(self):
        super(ComplexBBoxLoss, self).__init__()

    def forward(self, pred, target):
        """
        pred: (B, 4) complex tensor containing [x_pred, y_pred, h_pred, w_pred]
        target: (B, 4) tensor containing ground-truth [x, y, h, w] (real-valued)
        """

        # Convert target to complex (with 0 imaginary part)
        target_complex = torch.complex(target, torch.zeros_like(target))

        # Compute differences
        diff = target_complex - pred  # Directly subtract complex tensors

        # Loss: mean squared error on magnitudes of differences
        loss = torch.mean(torch.abs(diff)**2)
        return loss

# Example usage:
loss_fn = ComplexBBoxLoss()
# dummy predictions (complex) and targets (real)
pred_example = torch.complex(torch.tensor([[50.0, 30.0, 100.0, 200.0]]),
                             torch.tensor([[0.0, 0.0, 0.0, 0.0]]))
target_example = torch.tensor([[52.0, 29.0, 98.0, 205.0]])
print("Example loss:", loss_fn(pred_example, target_example).item())

Example loss: 8.5


In [60]:
import torchvision.transforms as transforms
from torchvision.datasets import VOCDetection  # Using VOC as an example; replace with COCO if needed
from utils import ToHSV, ToComplex

# Define transforms to match FCCNs input processing:
transform = transforms.Compose([
    transforms.ToTensor(),
    # Resize to the expected input size (e.g., 224×224 for ResNet)
    transforms.Resize((224, 224)),
    ToHSV(),
    ToComplex()
])

# As an example, we use the VOCDetection dataset (download it or use a custom dataset)
dataset = VOCDetection(root='.', image_set='train', download=True, transform=transform)

def detection_collate_fn(batch):
    """
    Custom collate function for detection tasks.
    Expects each element of the batch to be a tuple (image, target) where:
      - image is a tensor of shape (C, H, W) (uniform because of Resize)
      - target is the annotation (e.g., a list/dict with bounding boxes)
    Returns:
      - images: a stacked tensor of shape (B, C, H, W)
      - targets: a list of target annotations, one per image.
    """
    images = []
    targets = []
    for sample in batch:
        images.append(sample[0])
        targets.append(sample[1])

    images = torch.stack(images, 0)  # all images are same shape
    return images, targets

# Now, create your DataLoader with this collate function:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=2, collate_fn=detection_collate_fn)



In [61]:
for images, targets in dataloader:
    images = images.to(device)  # images are uniform

    # For demonstration, let's assume each target has a key 'annotation'
    # and within that an 'object' field containing a list of bounding boxes.
    # Here, we pick the first bounding box for each image.
    bbox_targets = []
    for t in targets:
        # Modify the extraction logic based on your annotation format.
        # For VOCDetection, it might be something like:
        objs = t['annotation'].get('object', [])
        if not isinstance(objs, list):
            objs = [objs]  # Ensure it's a list if only one object exists
        # Pick the first object's bounding box (x, y, width, height)
        # VOC annotations often store these as strings; convert to float.
        box = objs[0]['bndbox']
        x = float(box['xmin'])
        y = float(box['ymin'])
        w = float(box['xmax']) - float(box['xmin'])
        h = float(box['ymax']) - float(box['ymin'])
        bbox_targets.append([x, y, h, w])

    bbox_targets = torch.tensor(bbox_targets, dtype=torch.float32).to(device)

    # Proceed with forward pass, loss, backward, etc.
    optimizer.zero_grad()
    preds = detector(images)  # (B, 4) complex-valued predictions
    loss = loss_fn(preds, bbox_targets)
    loss.backward()
    optimizer.step()


In [63]:
# After training loop
save_path = "/content/FCCNs/detector_final.pth"
torch.save({
    'epoch': num_epochs,
    'model_state_dict': detector.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    # You can add additional items like loss history, hyperparameters, etc.
}, save_path)
print("Model saved to", save_path)


Model saved to /content/FCCNs/detector_final.pth


In [64]:
import torchvision.transforms as transforms
from torchvision.datasets import VOCDetection
from torch.utils.data import DataLoader
from utils import ToHSV, ToComplex

# Define the transform for validation (should match your training transforms except for random augmentations)
transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),  # Ensure same size as training images
    ToHSV(),
    ToComplex()
])

# Create the VOCDetection validation dataset
val_dataset = VOCDetection(root='.', image_set='val', download=True, transform=transform_val)

# Define a custom collate function for detection if you have variable-sized targets
def detection_collate_fn(batch):
    images = []
    targets = []
    for sample in batch:
        images.append(sample[0])
        targets.append(sample[1])  # Keep targets as a list (you'll process them later)
    images = torch.stack(images, 0)
    return images, targets

# Create the validation DataLoader
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2, collate_fn=detection_collate_fn)

print("Validation DataLoader created. Number of samples:", len(val_dataset))


Validation DataLoader created. Number of samples: 5823


In [65]:
detector.eval()
val_loss = 0.0
total_samples = 0

with torch.no_grad():
    for images, annotations in val_loader:
        images = images.to(device)
        # Process each annotation to extract bounding boxes (example for VOCDetection)
        bbox_targets = []
        for ann in annotations:
            # Adjust extraction logic based on annotation structure
            objs = ann['annotation'].get('object', [])
            if not isinstance(objs, list):
                objs = [objs]
            # Here, we simply pick the first object's bounding box as a dummy example
            box = objs[0]['bndbox']
            x = float(box['xmin'])
            y = float(box['ymin'])
            w = float(box['xmax']) - float(box['xmin'])
            h = float(box['ymax']) - float(box['ymin'])
            bbox_targets.append([x, y, h, w])

        bbox_targets = torch.tensor(bbox_targets, dtype=torch.float32).to(device)
        preds = detector(images)  # (B, 4) complex
        loss = loss_fn(preds, bbox_targets)
        val_loss += loss.item() * images.size(0)
        total_samples += images.size(0)

avg_val_loss = val_loss / total_samples
print("Average validation loss:", avg_val_loss)


Average validation loss: 22746.676598322385
